# Playground:  Segmentation workflow for RNA

This notebook contains the workflow to detect single-molecule RNA FISH signals that works well for us. It is based off the Allen Cell Segmenter workflow for sialyltransferase 1. 

Key steps of the workflows:

* Auto-Contrast intensity normalization
* Background subtraction
* Edge-reserving smoothing
* 3D Spot filter 
* Watershed algorithm to split adjacent objects
* Size thresholding

In [1]:
# IO packages
from aicsimageio import AICSImage
import os
import imageio

# calculation packages
import numpy as np

# visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# segmentation packages
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from aicssegmentation.core.seg_dot import dot_3d_wrapper
from skimage.morphology import dilation, ball, remove_small_objects

# watershed packages
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy.ndimage import distance_transform_edt
from skimage.measure import label


## Loading the data

We'll start by investigating the segmentation for a single image. You can change which image you're investigating using the FILE_NAME variable below.


In [2]:
# Update the path to your data
# For those on a Mac system, you can update your username


FILE_PATH = '/data/rna/raw-data/'

FILE_NAME = 'NC12_interphase_Slide22_Emb21_Img1.tif'

reader = AICSImage(FILE_PATH + FILE_NAME) 
IMG = reader.data


## Preview of the image

In [5]:
#####################
structure_channel = 0
#####################

structure_img0 = IMG[0, 0, structure_channel,:,:,:]


In [6]:
view(single_fluorescent_view(structure_img0))

Viewer(rendered_image=<itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x7f84c9…

## Image segmentation

### Step 1: Pre-Processing

About selected algorithms and tuned parameters

* **Intensity normalization**: Parameter `intensity_scaling_param` has two options: two values, say `[A, B]`, or single value, say `[K]`. For the first case, `A` and `B` are non-negative values indicating that the full intensity range of the stack will first be cut-off into **[mean - A * std, mean + B * std]** and then rescaled to **[0, 1]**. The smaller the values of `A` and `B` are, the higher the contrast will be. For the second case, `K`>0 indicates min-max Normalization with an absolute intensity upper bound `K` (i.e., anything above `K` will be chopped off and reset as the minimum intensity of the stack) and `K`=0 means min-max Normalization without any intensity bound.

    * Parameter for st6gal1:  `intensity_scaling_param = [9, 19]`


* **Smoothing**: 3D gaussian smoothing with `gaussian_smoothing_sigma = 1`. The large the value is, the more the image will be smoothed. 

In [7]:
################################
# First, calculate the best intensity normalization parameters

minimum_value = np.amin(structure_img0)
mean_value = np.mean(structure_img0)
percentile_99 = np.percentile(structure_img0, 99.99)
std_array = np.std(structure_img0)

a = round((mean_value - minimum_value) / std_array, 1)
b = round((percentile_99 - mean_value) / std_array, 1)


In [8]:
################################
## PARAMETER ##
intensity_scaling_param = [a, b]
gaussian_smoothing_sigma = 0.5
################################

# intensity normalization
structure_img = intensity_normalization(structure_img0, scaling_param=intensity_scaling_param)

# smoothing with gaussian filter
structure_img_smooth = image_smoothing_gaussian_slice_by_slice(structure_img, sigma=gaussian_smoothing_sigma)

In [9]:
# quickly visualize the image after smoothing
view(single_fluorescent_view(structure_img_smooth))

Viewer(rendered_image=<itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x7f84c8…

#### If the contrast looks too off, you can tune the normalization parameters.

The Allen Cell Segmenter has a function to give you some suggestions for the normalization parameters. If you have certain preference, you can adjust the values based on the suggestion.

***After you decide the parameters, you have to re-run the code above with the new parameter*** `intensity_scaling_param = ` 

### Step 2: Core Algorithm

#### step 2.1: Apply 3D Spot filter (S3)

Parameter syntax: `[[scale_1, cutoff_1], [scale_2, cutoff_2], ....]` 
* `scale_x` is set based on the estimated radius of your target dots. For example, if visually the diameter of the dots is usually 3~4 pixels, then you may want to set `scale_x` as `1` or something near `1` (like `1.25`). Multiple scales can be used, if you have dots of very different sizes.  
* `cutoff_x` is a threshold applied on the actual filter reponse to get the binary result. Smaller `cutoff_x` may yielf more dots and fatter segmentation, while larger `cutoff_x` could be less permisive and yield less dots and slimmer segmentation. 


In [10]:
################################
## PARAMETERS for this step ##
s3_param = [[1, 0.005], [10, 0.1]]
################################

bw = dot_3d_wrapper(structure_img_smooth, s3_param)


In [11]:
# view the segmentation result
viewer_bw = view(segmentation_quick_view(bw))
viewer_bw

Viewer(rendered_image=<itk.itkImagePython.itkImageUC3; proxy of <Swig Object of type 'itkImageUC3 *' at 0x7f84…

##### After quickly visualizing the segmentation results, you can also visualize the segmentation and original image side by side
##### You may select an ROI to inspect the details

* Option 1: Easy ROI selection, but NOT recommended if you are using a laptop

You can select an ROI in above visualization ('viewer_bw'); otherwise, the default ROI is the full image

[See this video for How to select ROI](https://www.youtube.com/watch?v=ZO8ey6-tF_0&index=3&list=PL2lHcsoU0YJsh6f8j2vbhg2eEpUnKEWcl)

* Option 2: Manually type in ROI coordinates

Type in the coordinates of upper left corner and lower right corner of the ROI in the form of [Upper_Left_X, Upper_Left_Y, Lower_right_X, Lower_right_Y]. 

In [12]:
# Option 1:
view(seg_fluo_side_by_side(structure_img0,bw,roi=['ROI',viewer_bw.roi_slice()]))

# Option 2: 
# view(seg_fluo_side_by_side(structure_img0,bw,roi=['M',[570,370,730,440]]))

Viewer(rendered_image=<itk.itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkImageF3 *' at 0x7f84c8…

##### If the above segmentation is satisfactory? Here are some possible things to check:

-------
* Is there big missing chunk? Or are segmented chunks are significantly fatter? You may visualize the intermediate result, i.e. the objects, by `view(segmentation_quick_view(object_for_debug))`. By doing this, you can have some sense whether the objects are roughly regions in individual cells. In other words, we want to roughly isolate the stuffs in individual cells. If not, you may change `global_thresh_method`. Three options `'tri'`, `'med'`,`'ave'` are implemented. `'tri'` is triangle method, `'med'` is median method, `'ave'` is the average of the values returned by triangle method and median method. 
* Observing missing chunks may be also due to falsely removed objects. Try to decrease `object_minArea` to be more permisive in segmenting objects.
* Do you observe a chunk of background stuffs in the segmentation? Try to increase `object_minArea` to exclude these background noise. 
* If you observe the segmented objects are slightly fatter than the actual size (may take defraction of light into consideration), don't worry, Next step (2.2) can help the make them thinner. 
* If you observe missing dots in the segmentation, don't worry. Later step (2.3) can pick them up.
--------

#### If the results are satisfactory, go to Step 2.2 directly; otherwise, try to tweak the parameters based on the above suggestions. 

Assumption: the segmentation result is saved in a variable named `bw`.

#### step 2.2: Use watershed algorithm to separate touching dots

In [13]:

minArea = 20

local_maxi = peak_local_max(structure_img0, min_distance=5, labels=label(bw), indices=False)
distance = distance_transform_edt(bw)
im_watershed = watershed(-distance, label(dilation(local_maxi, selem=ball(1))), mask=bw, watershed_line=True)


#### Step 3: Post-Processing 
Remove all objects smaller than the defined minimum area

In [ ]:
################################
## PARAMETERS for removing small objects ##
minArea = 20
################################

final_seg = remove_small_objects(im_watershed>0, min_size=minArea, connectivity=1, in_place=False)


## Result inspection

In [ ]:
viewer_final = view(segmentation_quick_view(final_seg))
viewer_final

### You can also focus your inspection on a small ROI

* Option 1: Easy ROI selection, but NOT recommended if you are using a laptop

You can select an ROI in above visualization ('viewer_final'); otherwise, the default ROI is the full image

[See this video for How to select ROI](https://www.youtube.com/watch?v=ZO8ey6-tF_0&index=3&list=PL2lHcsoU0YJsh6f8j2vbhg2eEpUnKEWcl)

* Option 2: Manually type in ROI coordinates

Type in the coordinates of upper left corner and lower right corner of the ROI in the form of [Upper_Left_X, Upper_Left_Y, Lower_right_X, Lower_right_Y]. 

In [ ]:
# Option 1: 
view(seg_fluo_side_by_side(structure_img0, final_seg, roi=['ROI',viewer_final.roi_slice()]))

# Option 2: 
# view(seg_fluo_side_by_side(struct_img, seg, roi=['M',[267,474, 468, 605]]))

### You may also physically save the segmentation results into a .tiff image

In [ ]:
# define where to save your test segmentations

output_filepath = '/output/test-segmentations/'

if not os.path.isdir(output_filepath):
    os.makedirs(output_filepath)


In [ ]:
# this file will be saved within your docker container volume "output"
# in order to visualize this most easily, you can copy this to your computer using
# docker cp jupyter:/output/ output/ 

output_seg = final_seg>0
out=output_seg.astype(np.uint8)
out[out>0]=255
imageio.volwrite(output_filepath + FILE_NAME + '-test_rna_seg.tiff', out)
